# Reading WSIS database

In [ ]:
import pandas as pd

In [ ]:
attributes = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_attributes.tsv', sep='\t')
profiles = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_profiles.tsv', sep='\t')
layers_chemical = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_layers_chemical.tsv', sep='\t')
layers_physycal = pd.read_csv('../data/WoSIS_2019_September/wosis_201909_layers_physical.tsv', sep='\t')

ORGC is the code for organic carbon

In [ ]:
attributes.iloc[23]

Use the code "ORGC" to look at the values:

In [ ]:
layers_chemical.orgc_value